# Intro to Recurrent Neural Networks (RNN)

This lab was built by Jon Barker, Solutions Architect, July 2016

This lab is aimed at Deep Learning (DL) beginners that are interested in seeing recurrent neural networks (RNNs) in action and gaining an intuition for how they work and how they are used.  This should not be your first exposure to neural networks and DL.  If it is, stop now and go complete our Intro to DL series of labs to learn the basics.

Before completing this lab, we also recommend you read the Parallel Forall blog post "Deep Learning in a Nutshell: Sequence Learning" - you can find it [here](https://devblogs.nvidia.com/parallelforall/deep-learning-nutshell-sequence-learning/#more-6437).  That post gives a gentle introduction to the basic idea behind Recurrent Neural Networks (RNNs) and in this lab you will get hands-on experience with some of the RNNs described in the article.

In this lab we will use both the Python and Lua programming languages.  If you are not familiar with Lua you can complete out "Intro to Torch7" lab to familiarize yourself with the basics.

## What are RNNs?

RNNs are an extension of regular artificial neural networks that add connections feeding the hidden state of the neural network back into itself - these are called recurrent connections.

The reason for adding these recurrent connections is to provide the network with visibility not just of the current data sample it has been provided, but also it's previous hidden state.  In some sense, this gives the network a sequential memory of what it has seen before.  

This makes RNNs applicable in situations where a sequence of data is required to make a classification decision or regression estimate.  For example, you might employ an RNN to predict your future household electricity consumption based on the time-series of instantaneous power draw or you might use an RNN to predict the next word that a person will utter given the previous ten they have said.

# Example 1: Binary addition

**Credit**:  This first example is adapted with permission from [this](https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/) excellent blog post by Andrew Trask [@iamtrask](https://twitter.com/iamtrask).

Further explanation of the same example can be found in [this](https://www.youtube.com/watch?v=z61VFeALk3o) video by Geoffrey Hinton.

## The problem

This toy problem is chosen to demonstrate something you can do with a RNN that you cannot do easily with a regular feed-forward neural network.  The problem is adding up two binary numbers.  For example consider the following problem:

![Binary addition](files/binary_addition_small.png)

If you don't know how binary addition works, read [this](http://web.math.princeton.edu/math_alive/1/Lab1/BinAdd.html) first.  

## Neural network solutions

We could train a regular feed-forward (fully-connected) neural network to do binary addition.  For example, we could give it the following structure:

![Binary addition FFNN](files/binary_addition_FF.png)

This approach has some obvious limitations though:

- We have to decide in advance and fix the maximum number of digits in each binary number.
- The knowledge learned about adding digits at the beginning of the binary numbers does not get generalized and applied to digits at the end.  We have to learn different weights describing how to add digits at all locations.

The RNN we are going to train has a very similar structure to a fully-connected neural network:

![Binary addition RNN](files/binary_addition_RNN.png)

We have changed how the network receives input.  Instead of taking in the two whole binary numbers, we just take in one digit at a time from each number and try to predict the corresponding digit in the sum.  It is the job of the RNN to remember whether we have a carried bit from an overflow when we calculated the sum at the last digit.  

We have also added connections between the hidden units themselves in addition to the connections between the hidden units and the input and output units.  These additional connections mean that the hidden activity at one time step can influence the hidden activity at the next time step in addition to the influence from the input units at the next time step.  You might ask why we don't just pass the output at the previous time-step as an additional input at the next time-step instead.  Technically you could do this, but you would be missing out on the knowledge (or memory) that the hidden layer can accumulate over multiple time-steps and you would just be influenced by the last time-step alone.

In [2]:
# Import some dependencies
import copy
import numpy as np

# Fixed random seed for reproducability
np.random.seed(0)

In [3]:
# define sigmoid nonlinearity
def sigmoid(x):
    return 1/(1+np.exp(-x))

# define the derivative of the sigmoid
# this is needed for backpropagating gradients
def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [4]:
# Generate a lookup table mapping integers to binary representations
int2binary = {}
binary_dim = 8 # The maximum length of the binary numbers we'll be adding
# Note: we don't technically need to fix the maximum length of the binary numbers to train our RNN
# We are just doing it for convenience of creating our dataset here

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]
    
print "The binary representation of 53 is : " + str(int2binary[53])

The binary representation of 53 is : [0 0 1 1 0 1 0 1]


In [5]:
input_dim = 2
hidden_dim = 16 
# See the Geoffrey Hinton video linked above for a great explanation why we actually only need 3 hidden units
# We only require three hidden units in our network because of the small number of required states 
# See the Geoffrey Hinton video above for a complete explanation of this
# We are using 16 units here to speed up training
output_dim = 1

# Randomly initialize neural network weights
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1  # Weight matrix between inputs and hidden units
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1 # Weight matrix between hidden units and output units
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1 # Recurrence weight matrix between hidden units

synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [6]:
# generate a random addition problem (a + b = c)
a_int = np.random.randint(largest_number/2) # int version
a = int2binary[a_int] # binary encoding
print "a = " + str(a) + "\n"

b_int = np.random.randint(largest_number/2) # int version
b = int2binary[b_int] # binary encoding
print "b = " + str(b) + "\n"

# Directly calculate the true answer as our label
c_int = a_int + b_int
c = int2binary[c_int]
print "c (Truth) = " + str(c) + "\n"
    
# This is where we'll store our (binary encoded) prediction
d = np.zeros_like(c)

layer_1_values = list()
layer_1_values.append(np.zeros(hidden_dim))

for position in range(binary_dim):
        
        # generate input and output
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
print "d (Prediction) = " + str(d)

a = [0 0 0 0 1 0 0 1]

b = [0 0 1 1 1 1 0 0]

c (Truth) = [0 1 0 0 0 1 0 1]

d (Prediction) = [0 0 0 0 0 0 0 1]


## RNN training

The algorithm we will use to train our RNN is the same stochastic gradient descent (SGD) algorithm you have seen in training multi-layer perceptrons and convolutional neural networks.  The only difference is in the backpropagation step that computes the weight updates for our slightly more complex network structure.  Study this excellent animation from [@iamtrask's blog](https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/) of an RNN being trained on a four step sequence (note that the network architecture is slightly different from our example, but the training process is the same):

![Backpropagation through time (BPTT)](files/BPTT.gif)

As information propagates from the input neurons to the hidden neurons over the four timesteps you see that the hidden units become a combined representation of the four inputs.  Once all of the timesteps have been received the output neurons turn black, meaning that the predicted output digits are generated.  The output neurons then turn yellow, meaning that the error in those predictions is calculated.  Then comes the crucial part:  starting at the last output neuron the error gradient is computed and backpropagated to the hidden units for that time-step (the neurons go orange) and this process is also repeated for each of the previous time-steps in order.  Remember when the gradients backpropagate to the hidden units they are coming from both the output neurons and the hidden units one step ahead in the sequence.  We call this process Backpropagation Through Time (BPTT).

Notice that this diagram clearly shows that you can think of an RNN that has been "unrolled" back through all the time steps it has processed simply as a regular feed-forward neural network trained with backpropagation but with a slightly more complicated network structure.

## (Finally) the training code

Now work through the code for training the RNN.  Do not worry too much about understanding all the details, but try to understand the overall flow of a training iteration.

In [7]:
# Training algorithm parameters
alpha = 0.1   # This is the learning rate  

In [8]:
# Training loop - 10000 iterations
for j in range(10000):
    
    # generate a random addition problem (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding

    # Directly calculate the true answer as our label
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    # This is where we'll store our (binary encoded) prediction
    d = np.zeros_like(c)

    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    
    # Feed-forward iterating along the digits in the binary encodings
    for position in range(binary_dim):
        
        # generate input and output
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))

        # did we miss?... if so, by how much?
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # store hidden layer so we can use it in the next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
    
    future_layer_1_delta = np.zeros(hidden_dim)
    
    # BPTT starting at the end of the binary encodings and iterating backwards
    for position in range(binary_dim):
        
        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        # error at hidden layer
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) 
                         + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)

        # let's update all our weights so we can try again
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    
    # SGD update equations
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # print out progress
    if(j % 500 == 0):
        print "Error:" + str(overallError)
        print "Pred:" + str(d)
        print "True:" + str(c)
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        if a_int + b_int == out:
            correct = "TRUE"
        else:
            correct = "FALSE"
        print str(a_int) + " + " + str(b_int) + " = " + str(out) + " " + correct
        print "------------"

Error:[ 3.94471861]
Pred:[0 0 0 0 0 0 0 0]
True:[1 1 0 0 1 0 1 0]
91 + 111 = 0 FALSE
------------
Error:[ 3.89847019]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 1 1 0 1 1]
10 + 17 = 0 FALSE
------------
Error:[ 3.83932294]
Pred:[1 1 1 1 1 1 1 1]
True:[0 1 0 0 1 1 1 1]
53 + 26 = 255 FALSE
------------
Error:[ 3.79387727]
Pred:[1 1 1 1 1 1 1 1]
True:[0 1 1 1 1 1 1 1]
6 + 121 = 255 FALSE
------------
Error:[ 4.14569876]
Pred:[0 0 0 0 0 0 0 0]
True:[1 1 0 1 0 1 1 0]
90 + 124 = 0 FALSE
------------
Error:[ 3.79283125]
Pred:[0 1 1 0 1 1 1 0]
True:[0 1 1 0 1 0 0 0]
82 + 22 = 110 FALSE
------------
Error:[ 4.00058902]
Pred:[1 1 1 1 1 0 1 1]
True:[1 1 1 0 1 1 0 0]
109 + 127 = 251 FALSE
------------
Error:[ 2.99012715]
Pred:[0 0 1 1 1 0 0 1]
True:[0 0 1 1 1 0 0 1]
44 + 13 = 57 TRUE
------------
Error:[ 3.8681077]
Pred:[1 1 0 0 0 1 1 0]
True:[1 0 1 1 0 1 0 0]
123 + 57 = 198 FALSE
------------
Error:[ 2.96698262]
Pred:[1 0 0 1 1 0 0 0]
True:[1 0 0 1 1 1 1 0]
71 + 87 = 152 FALSE
------------
Error:[ 1.5491

As you see the network very quickly learns to correctly add the binary numbers in this simple example.

In the next part of the lab we will explore the challenges of training RNNs for real-world datasets as some of the advances that have been made in addressing these challenges.  To open Part 2 of the lab click [here](Introduction%20to%20RNNs%20-%20notebook%202%20of%202.ipynb#)